In [11]:
from models.MLP_dir import MLP_dir
from models.MLP_incr import MLP_incr
from data.materials import load_responses
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import os.path as osp

def data_to_tensor(x:np.array):
    return torch.tensor(
        x.astype(np.float32),
        dtype=torch.float32
    ).unsqueeze(-1)

In [12]:
def train_model(model: MLP_dir | MLP_incr, mat_name, inp_name): 
    eps_list, sig_list = load_responses(mat_name,'random',inp_name,data_path='data')

    y_list = data_to_tensor(sig_list)
    u_list = data_to_tensor(eps_list)

    y_train, y_tmp, u_train, u_tmp = train_test_split(y_list, u_list, test_size=0.3, random_state=42)
    y_val, y_test, u_val, u_test = train_test_split(y_tmp, u_tmp, test_size=0.5, random_state=42)

    model.fit(
        epochs=500,
        y_train=y_train, u_train=u_train,
        y_val=y_val,u_val=u_val,
        early_stopping=True,
        patience=20,
        min_delta=1e-6,
        verbose=True,
        shuffle=False,
    )

    return model.evaluate(y_test,u_test).item()

In [13]:
model = MLP_incr(7,8,3,seed=56)

test_err = train_model(
    model,
    mat_name = 'isotropic-swift',
    inp_name = 'pd_ms_42_200',
)

print(f"{test_err*100} %")

Epoch 1/500, Loss: 0.009756211191415787, Val Loss: 0.01663209404796362, LR: 0.001
Epoch 2/500, Loss: 0.00525799673050642, Val Loss: 0.013895776254745822, LR: 0.001
Epoch 3/500, Loss: 0.0036581687163561583, Val Loss: 0.011724650130296747, LR: 0.001
Epoch 4/500, Loss: 0.002511413535103202, Val Loss: 0.010388251638505607, LR: 0.001
Epoch 5/500, Loss: 0.0020453459583222866, Val Loss: 0.009440997363223383, LR: 0.001
Epoch 6/500, Loss: 0.0017893933691084385, Val Loss: 0.008525699280047169, LR: 0.001
Epoch 7/500, Loss: 0.001782107981853187, Val Loss: 0.007805441653666397, LR: 0.001
Epoch 8/500, Loss: 0.0018346132710576057, Val Loss: 0.007161235015761728, LR: 0.001
Epoch 9/500, Loss: 0.0017901296960189939, Val Loss: 0.006545661449975645, LR: 0.001
Epoch 10/500, Loss: 0.0016613274347037077, Val Loss: 0.006014373549260199, LR: 0.001
Epoch 11/500, Loss: 0.0014497463125735521, Val Loss: 0.005426642434516301, LR: 0.001
Epoch 12/500, Loss: 0.0012935030972585082, Val Loss: 0.004899725300492719, LR: 0

In [14]:
def eval_model(model: MLP_dir | MLP_incr, mat_name, inp_type, inp_name, save_plot=False):
    
    eps_list, sig_list = load_responses(
        mat_name,inp_type,inp_name,
        data_path='data'
    )

    y_list = data_to_tensor(sig_list)
    u_list = data_to_tensor(eps_list)

    return model.evaluate(
        y_list,u_list,
        save_plot=save_plot,
        path=osp.join(f'tmp/{inp_name}.png') # TODO: finish path
    ).item()

In [15]:
mat_name = 'isotropic-swift'

eval_list = []
for inp_name in ['amplitude', 'cyclic', 'impulse', 'piecewise']:
    eval_list += [
        eval_model(model,mat_name,'static',inp_name,save_plot=True)
    ]
    print(f'{inp_name}: {eval_list[-1]*100:02f} %')

for inp_name in ['bl_ms_42_200', 'gp_42_200', 'pd_ms_42_200', 'rw_42_200']:
    eval_list += [
        eval_model(model,mat_name,'random',inp_name,save_plot=False)
    ]
    print(f'{inp_name}: {eval_list[-1]*100:02f} %')

print(f'\nMean: {np.mean(eval_list)*100:02f} %')

amplitude: 3.337774 %
cyclic: 40.973404 %
impulse: 5.221648 %
piecewise: 5.276799 %
bl_ms_42_200: 0.942825 %
gp_42_200: 1.106047 %
pd_ms_42_200: 0.623593 %
rw_42_200: 1.310693 %

Mean: 7.349098 %


(0, 200, False): 3.656732 %

(1e-4, 200, False): 3.622211 %

(1e-4, 200, True): 3.656732 %

(1e-2, 200, False): 4.618025 %

(1e-4, 1000, False): 64.602013 %

(1e-2, 1000, False): 6.192588 %